In [ ]:
import json

# Read potentially malformed JSON from a file
with open("D:\\Project\\pbix\\Human Resources Sample PBIX\\Report\\Layout", 'r') as f:
    raw = f.read()

# Remove null bytes if present
raw = raw.replace('\x00', '')

# Try to parse JSON safely
try:
    data = json.loads(raw)
except json.JSONDecodeError as e:
    print(f"JSONDecodeError: {e}")
    # Optionally, handle or repair the JSON here
    # For demo purposes, exit if error
    data = None

# If parsing succeeded, save cleaned JSON to a new file
if data is not None:
    with open("D:\\Project\\pbix\\data.json", 'w') as f:
        json.dump(data, f, indent=4)
    print("Cleaned JSON saved to cleaned.json")
else:
    print("Could not parse JSON after cleanup.")

Cleaned JSON saved to cleaned.json


In [ ]:
import json

# Read raw content from input.json
with open("D:\\Project\\pbix\\Human Resources Sample PBIX\\Report\\Layout", 'r') as f:
    raw = f.read().replace('\x00', '').strip()

# First decode: Remove outer quotes if needed
if raw.startswith('"') and raw.endswith('"'):
    # Unescape and decode the string inside the quotes
    intermediate = json.loads(raw)
else:
    intermediate = raw

# Second decode: Convert to Python dict
if isinstance(intermediate, str):
    try:
        data = json.loads(intermediate)
    except json.JSONDecodeError as e:
        print(f"JSONDecodeError: {e}")
        data = None
else:
    data = intermediate

# Save cleaned JSON to cleaned.json
if data is not None:
    with open("D:\\Project\\pbix\\data.json", 'w') as f:
        json.dump(data, f, indent=4)
    print("Cleaned JSON saved to cleaned.json")
else:
    print("Could not parse JSON after cleanup.")

Cleaned JSON saved to cleaned.json


In [2]:
inputfile="D:\\Project\\pbix\\Human Resources Sample PBIX\\Report\\Layout"
outfile="D:\\Project\\pbix\\data2.json"

In [5]:
import json

def parse_json_content(raw):
    cleaned = raw.replace('\x00', '').strip()
    try:
        first = json.loads(cleaned)
    except json.JSONDecodeError:
        return None
    if isinstance(first, str):
        try:
            second = json.loads(first)
            return second
        except json.JSONDecodeError:
            return None
    else:
        return first

# Safest way: read as bytes, decode with replacement for errors
with open(inputfile, 'rb') as f:
    raw_bytes = f.read()
raw = raw_bytes.decode('utf-8', errors='replace')

parsed = parse_json_content(raw)

if parsed is not None:
    with open(outfile, 'w', encoding='utf-8') as f:
        json.dump(parsed, f, indent=4)
    print("Cleaned JSON saved to cleaned.json")
else:
    print("Could not parse JSON after cleanup.")

Cleaned JSON saved to cleaned.json


In [ ]:
import json
import pandas as pd
import zipfile
import os
import tempfile
# import json

def unzip_pbix(pbix_path):
    """
    Unzips a Power BI .pbix file to a temporary directory.

    Args:
        pbix_path (str): Path to the .pbix file.

    Returns:
        str: Path to the temporary directory containing the extracted files.
    """
    temp_dir = tempfile.mkdtemp()
    with zipfile.ZipFile(pbix_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    return temp_dir

def find_layout_json(unzipped_dir):
    """
    Finds the Layout file in the unzipped PBIX directory.

    Args:
        unzipped_dir (str): Path to the directory with PBIX contents.

    Returns:
        str: Full path to the Layout JSON file.

    Raises:
        FileNotFoundError: If no Layout file is found.
    """
    # Usually named "Layout" or "Layout.json"
    for fname in os.listdir(unzipped_dir):
        if fname.lower().startswith("layout"):
            layout_path = os.path.join(unzipped_dir, fname)
            return layout_path
    raise FileNotFoundError("Layout JSON not found in PBIX contents.")

def read_layout_json(layout_path):
    """
    Reads and parses the Layout JSON file from PBIX contents.

    Args:
        layout_path (str): Path to the Layout JSON file.

    Returns:
        dict: Parsed JSON content of the Layout file.
    """
    with open(layout_path, 'rb') as f:
        raw_bytes = f.read()
    raw_str = raw_bytes.decode('utf-8', errors='replace').replace('\x00', '').strip()
    return json.loads(raw_str)

def extract_alias_map(proto_query):
    alias_map = {}
    from_list = proto_query.get('From', [])
    for entry in from_list:
        alias = entry.get('Name')
        entity = entry.get('Entity')
        if alias and entity:
            alias_map[alias] = entity
    return alias_map

def extract_fields(selects, alias_map):
    columns = []
    measures = []
    tables = set()
    if not isinstance(selects, list):
        return columns, measures, tables
    for sel in selects:
        # Column reference
        if 'Column' in sel:
            col = sel['Column']
            source = None
            if 'Expression' in col and 'SourceRef' in col['Expression']:
                source = col['Expression']['SourceRef'].get('Source')
            column = col.get('Property')
            full_table = alias_map.get(source, source)
            if full_table and column:
                columns.append(f"{full_table}.{column}")
                tables.add(full_table)
        # Measure reference
        if 'Measure' in sel:
            meas = sel['Measure']
            source = None
            if 'Expression' in meas and 'SourceRef' in meas['Expression']:
                source = meas['Expression']['SourceRef'].get('Source')
            measure = meas.get('Property')
            full_table = alias_map.get(source, source)
            if full_table and measure:
                measures.append(f"{full_table}.{measure}")
                tables.add(full_table)
        # Aggregation reference
        if 'Aggregation' in sel:
            agg = sel['Aggregation']
            function_map = {
                0: 'Sum', 1: 'Count', 2: 'Min', 3: 'Max', 4: 'Average', 5: 'DistinctCount'
            }
            func_name = function_map.get(agg.get('Function'), 'Aggregation')
            agg_expr = agg.get('Expression', {})
            if 'Column' in agg_expr:
                col = agg_expr['Column']
                source = None
                if 'Expression' in col and 'SourceRef' in col['Expression']:
                    source = col['Expression']['SourceRef'].get('Source')
                column = col.get('Property')
                full_table = alias_map.get(source, source)
                if full_table and column:
                    measures.append(f"{func_name}({full_table}.{column})")
                    tables.add(full_table)
    return columns, measures, tables

def extract_filter_columns(filters_json, alias_map):
    filter_columns = []
    if not filters_json:
        return filter_columns
    try:
        filters = json.loads(filters_json)
    except Exception:
        return filter_columns
    for f in filters:
        expr = f.get('expression', {})
        # Only handle column filters
        if 'Column' in expr:
            col = expr['Column']
            entity = None
            if 'Expression' in col and 'SourceRef' in col['Expression']:
                entity = col['Expression']['SourceRef'].get('Entity')
            elif 'SourceRef' in col:
                entity = col['SourceRef'].get('Entity')
            column = col.get('Property')
            if not entity:
                # Try to get Source and map to Entity
                source = None
                if 'Expression' in col and 'SourceRef' in col['Expression']:
                    source = col['Expression']['SourceRef'].get('Source')
                elif 'SourceRef' in col:
                    source = col['SourceRef'].get('Source')
                entity = alias_map.get(source, source)
            if entity and column:
                filter_columns.append(f"{entity}.{column}")
    return filter_columns

def extract_visuals_from_section(section):
    results = []
    # Section-level filters
    section_filters = section.get('filters')
    # Try to get alias map from first visual (if any)
    alias_map = {}
    if section.get('visualContainers'):
        first_visual = section['visualContainers'][0]
        config_str = first_visual.get('config')
        try:
            config = json.loads(config_str)
            proto_query = config.get('singleVisual', {}).get('prototypeQuery', {})
            alias_map = extract_alias_map(proto_query)
        except Exception:
            pass
    section_filter_columns = extract_filter_columns(section_filters, alias_map)
    for idx, visual in enumerate(section.get('visualContainers', [])):
        config_str = visual.get('config')
        try:
            config = json.loads(config_str)
        except Exception as e:
            print(f"Error decoding config for visual {idx}: {e}")
            continue
        visual_type = config.get('singleVisual', {}).get('visualType')
        proto_query = config.get('singleVisual', {}).get('prototypeQuery', {})
        local_alias_map = extract_alias_map(proto_query)
        # Prefer local alias map, fallback to section alias map
        alias_map_used = local_alias_map if local_alias_map else alias_map
        selects = proto_query.get('Select', [])
        columns, measures, tables = extract_fields(selects, alias_map_used)
        # Visual-level filters
        visual_filters = visual.get('filters')
        visual_filter_columns = extract_filter_columns(visual_filters, alias_map_used)
        # Combine section and visual filters
        combined_filters = sorted(list(set(section_filter_columns + visual_filter_columns)))
        results.append({
            'visual_idx': idx,
            'visualType': visual_type,
            'tables': sorted(list(tables)),
            'columns': sorted(columns),
            'measures': sorted(measures),
            'filters': combined_filters
        })
    return results

def main(layout_path):
    with open(layout_path, 'rb') as f:
        raw_bytes = f.read()
    raw = raw_bytes.decode('utf-8', errors='replace').replace('\x00', '').strip()
    layout = json.loads(raw)
    for section in layout.get('sections', []):
        print(f"=== Section: {section.get('name')} ('{section.get('displayName')}') ===")
        visuals = extract_visuals_from_section(section)
        for visual in visuals:
            print(f"--- Visualization {visual['visual_idx']} ---")
            print(f"Type: {visual['visualType']}")
            print(f"Tables: {visual['tables']}")
            print(f"Columns: {visual['columns']}")
            print(f"Measures: {visual['measures']}")
            print(f"Filters: {visual['filters']}")
            print()

def visuals_to_dataframe(visuals):
    # Each cell as comma-separated string for tables/columns/measures/filters
    df = pd.DataFrame([{
        'Section': visual.get('section', ''),  # Add section if available
        'Visual Index': visual['visual_idx'],
        'Type': visual['visualType'],
        'Tables': ', '.join(visual['tables']),
        'Columns': ', '.join(visual['columns']),
        'Measures': ', '.join(visual['measures']),
        'Filters': ', '.join(visual.get('filters', [])),
    } for visual in visuals])
    return df

def main2(layout_path):
    with open(layout_path, 'rb') as f:
        raw_bytes = f.read()
    raw = raw_bytes.decode('utf-8', errors='replace').replace('\x00', '').strip()
    layout = json.loads(raw)
    for section in layout.get('sections', []):
        print(f"=== Section: {section.get('name')} ('{section.get('displayName')}') ===")
        visuals = extract_visuals_from_section(section)
        df = visuals_to_dataframe(visuals)
        print(df)
        # for visual in visuals:
        #     print(f"--- Visualization {visual['visual_idx']} ---")
        #     print(f"Type: {visual['visualType']}")
        #     print(f"Tables: {visual['tables']}")
        #     print(f"Columns: {visual['columns']}")
        #     print(f"Measures: {visual['measures']}")
        #     print(f"Filters: {visual['filters']}")
        #     print()



In [46]:
 # Replace with your PBIX file path
pbix_path = 'D:\\Project\\pbix\\Human Resources Sample PBIX.pbix'
infile ="D:\\Project\\pbix\\Layout.json"

main2(infile)

=== Section: ReportSection1 ('Info') ===
  Section  Visual Index     Type Tables Columns Measures Filters
0                     0    image                                
1                     1  textbox                                
=== Section: ReportSection2 ('New Hires') ===
  Section  Visual Index                           Type  \
0                     0                        textbox   
1                     1                      lineChart   
2                     2  lineClusteredColumnComboChart   
3                     3                       pieChart   
4                     4                 waterfallChart   
5                     5                        textbox   
6                     6    lineStackedColumnComboChart   
7                     7                        textbox   
8                     8                        textbox   

                    Tables                                Columns  \
0                                                                   

In [65]:
import json
import pandas as pd
# import json
import pandas as pd
import zipfile
import os
import tempfile
# import json

def unzip_pbix(pbix_path):
    """
    Unzips a Power BI .pbix file to a temporary directory.

    Args:
        pbix_path (str): Path to the .pbix file.

    Returns:
        str: Path to the temporary directory containing the extracted files.
    """
    temp_dir = tempfile.mkdtemp()
    with zipfile.ZipFile(pbix_path, 'r') as zip_ref:
        zip_ref.extractall(temp_dir)
    return temp_dir

def find_layout_json(unzipped_dir):
    """
    Finds the Layout file in the unzipped PBIX directory (case-insensitive), regardless of extension or subfolder.

    Args:
        unzipped_dir (str): Path to the directory with PBIX contents.

    Returns:
        str: Full path to the Layout JSON file.

    Raises:
        FileNotFoundError: If no Layout file is found.
    """
    for root, dirs, files in os.walk(unzipped_dir):
        for fname in files:
            if fname.lower().startswith("layout"):
                layout_path = os.path.join(root, fname)
                return layout_path
    raise FileNotFoundError("Layout file not found in PBIX contents.")

def read_layout_json(layout_path):
    """
    Reads and parses the Layout JSON file from PBIX contents.

    Args:
        layout_path (str): Path to the Layout JSON file.

    Returns:
        dict: Parsed JSON content of the Layout file.
    """
    with open(layout_path, 'rb') as f:
        raw_bytes = f.read()
    raw_str = raw_bytes.decode('utf-8', errors='replace').replace('\x00', '').strip()
    return json.loads(raw_str)

def extract_alias_map(proto_query):
    alias_map = {}
    from_list = proto_query.get('From', [])
    for entry in from_list:
        alias = entry.get('Name')
        entity = entry.get('Entity')
        if alias and entity:
            alias_map[alias] = entity
    return alias_map

def extract_alias_map(proto_query):
    alias_map = {}
    from_list = proto_query.get('From', [])
    for entry in from_list:
        alias = entry.get('Name')
        entity = entry.get('Entity')
        if alias and entity:
            alias_map[alias] = entity
    return alias_map

def extract_fields(selects, alias_map):
    columns = []
    measures = []
    tables = set()
    if not isinstance(selects, list):
        return columns, measures, tables
    for sel in selects:
        # Column reference
        if 'Column' in sel:
            col = sel['Column']
            source = None
            if 'Expression' in col and 'SourceRef' in col['Expression']:
                source = col['Expression']['SourceRef'].get('Source')
            column = col.get('Property')
            full_table = alias_map.get(source, source)
            if full_table and column:
                columns.append(f"{full_table}.{column}")
                tables.add(full_table)
        # Measure reference
        if 'Measure' in sel:
            meas = sel['Measure']
            source = None
            if 'Expression' in meas and 'SourceRef' in meas['Expression']:
                source = meas['Expression']['SourceRef'].get('Source')
            measure = meas.get('Property')
            full_table = alias_map.get(source, source)
            if full_table and measure:
                measures.append(f"{full_table}.{measure}")
                tables.add(full_table)
        # Aggregation reference
        if 'Aggregation' in sel:
            agg = sel['Aggregation']
            function_map = {
                0: 'Sum', 1: 'Count', 2: 'Min', 3: 'Max', 4: 'Average', 5: 'DistinctCount'
            }
            func_name = function_map.get(agg.get('Function'), 'Aggregation')
            agg_expr = agg.get('Expression', {})
            if 'Column' in agg_expr:
                col = agg_expr['Column']
                source = None
                if 'Expression' in col and 'SourceRef' in col['Expression']:
                    source = col['Expression']['SourceRef'].get('Source')
                column = col.get('Property')
                full_table = alias_map.get(source, source)
                if full_table and column:
                    measures.append(f"{func_name}({full_table}.{column})")
                    tables.add(full_table)
    return columns, measures, tables

def extract_filter_columns(filters_json, alias_map):
    filter_columns = []
    if not filters_json:
        return filter_columns
    try:
        filters = json.loads(filters_json)
    except Exception:
        return filter_columns
    for f in filters:
        expr = f.get('expression', {})
        # Only handle column filters
        if 'Column' in expr:
            col = expr['Column']
            entity = None
            if 'Expression' in col and 'SourceRef' in col['Expression']:
                entity = col['Expression']['SourceRef'].get('Entity')
            elif 'SourceRef' in col:
                entity = col['SourceRef'].get('Entity')
            column = col.get('Property')
            if not entity:
                # Try to get Source and map to Entity
                source = None
                if 'Expression' in col and 'SourceRef' in col['Expression']:
                    source = col['Expression']['SourceRef'].get('Source')
                elif 'SourceRef' in col:
                    source = col['SourceRef'].get('Source')
                entity = alias_map.get(source, source)
            if entity and column:
                filter_columns.append(f"{entity}.{column}")
    return filter_columns

def extract_all_visuals(layout):
    visuals_all = []
    for section in layout.get('sections', []):
        section_name = section.get('name', '')
        section_display = section.get('displayName', '')
        section_filters = section.get('filters')
        # Try alias map from first visual (if any)
        alias_map = {}
        if section.get('visualContainers'):
            first_visual = section['visualContainers'][0]
            config_str = first_visual.get('config')
            try:
                config = json.loads(config_str)
                proto_query = config.get('singleVisual', {}).get('prototypeQuery', {})
                alias_map = extract_alias_map(proto_query)
            except Exception:
                pass
        section_filter_columns = extract_filter_columns(section_filters, alias_map)
        for idx, visual in enumerate(section.get('visualContainers', [])):
            config_str = visual.get('config')
            try:
                config = json.loads(config_str)
            except Exception as e:
                continue
            visual_type = config.get('singleVisual', {}).get('visualType')
            proto_query = config.get('singleVisual', {}).get('prototypeQuery', {})
            local_alias_map = extract_alias_map(proto_query)
            alias_map_used = local_alias_map if local_alias_map else alias_map
            selects = proto_query.get('Select', [])
            columns, measures, tables = extract_fields(selects, alias_map_used)
            visual_filters = visual.get('filters')
            visual_filter_columns = extract_filter_columns(visual_filters, alias_map_used)
            combined_filters = sorted(list(set(section_filter_columns + visual_filter_columns)))
            visuals_all.append({
                'Section': section_name,
                'Section Display': section_display,
                'Visual Index': idx,
                'Type': visual_type,
                'Tables': ', '.join(sorted(list(tables))),
                'Columns': ', '.join(sorted(columns)),
                'Measures': ', '.join(sorted(measures)),
                'Filters': ', '.join(combined_filters)
            })
    return visuals_all

def pbix_layout_to_df(layout_path):
    with open(layout_path, 'rb') as f:
        raw_bytes = f.read()
    raw = raw_bytes.decode('utf-8', errors='replace').replace('\x00', '').strip()
    layout = json.loads(raw)
    visuals = extract_all_visuals(layout)
    df = pd.DataFrame(visuals)
    return df

def explode_visuals_df(df):
    # Split comma fields into lists, then explode
    df1 = df.copy()
    # Ensure empty strings become empty lists
    for col in ["Tables", "Columns", "Measures", "Filters"]:
        df1[col] = df1[col].apply(lambda x: [i.strip() for i in x.split(",")] if x else [])
    # Explode Columns
    cols_df = df1.explode("Columns")
    cols_df = cols_df[cols_df["Columns"] != ""]
    cols_df["iscolumn/ismeasure"] = "iscolumn"
    cols_df["Columns/Measures"] = cols_df["Columns"]
    # Explode Measures
    meas_df = df1.explode("Measures")
    meas_df = meas_df[meas_df["Measures"] != ""]
    meas_df["iscolumn/ismeasure"] = "ismeasure"
    meas_df["Columns/Measures"] = meas_df["Measures"]
    # Combine
    out_df = pd.concat([cols_df, meas_df], ignore_index=True)
    # Explode Filters (optional: keep all filters, or join them with visual)
    out_df = out_df.explode("Filters")
    # Clean up columns
    out_df = out_df[["Section", "Section Display", "Visual Index", "Type", "Tables", "iscolumn/ismeasure", "Columns/Measures", "Filters"]]
    return out_df

def explode_df_one_table_per_row(df):
    rows = []
    for idx, row in df.iterrows():
        section = row.get("Section", "")
        section_display = row.get("Section Display", "")
        visual_idx = row.get("Visual Index", "")
        vtype = row.get("Type", "")
        filters = [f.strip() for f in str(row.get("Filters", "")).split(",") if f.strip()]
        
        # Explode columns
        for col in [c.strip() for c in str(row.get("Columns", "")).split(",") if c.strip()]:
            # Table is whatever is before the dot
            table = col.split(".", 1)[0] if "." in col else ""
            rows.append({
                "Section": section,
                "Section Display": section_display,
                "Visual Index": visual_idx,
                "Type": vtype,
                "Tables": table,
                "iscolumn/ismeasure": "iscolumn",
                "Columns/Measures": col,
                "Filters": ", ".join(filters)
            })
        # Explode measures
        for meas in [m.strip() for m in str(row.get("Measures", "")).split(",") if m.strip()]:
            # Table is whatever is before the dot (for Employee.New Hires) or inside parenthesis (for Sum(Employee.BadHires))
            # Handle both cases
            table = ""
            if "." in meas:
                # Format like Employee.New Hires or Sum(Employee.BadHires)
                if "(" in meas and ")" in meas:
                    # Find table inside parenthesis
                    import re
                    m = re.search(r"\(([^.]+)\.[^)]+\)", meas)
                    table = m.group(1) if m else ""
                else:
                    # Just split before dot
                    table = meas.split(".", 1)[0]
            rows.append({
                "Section": section,
                "Section Display": section_display,
                "Visual Index": visual_idx,
                "Type": vtype,
                "Tables": table,
                "iscolumn/ismeasure": "ismeasure",
                "Columns/Measures": meas,
                "Filters": ", ".join(filters)
            })
    out_df = pd.DataFrame(rows)
    return out_df

def explode_df_one_table_per_row2(df, pbix_filename):
    rows = []
    for idx, row in df.iterrows():
        section = row.get("Section", "")
        section_display = row.get("Section Display", "")
        visual_idx = row.get("Visual Index", "")
        vtype = row.get("Type", "")
        filters = [f.strip() for f in str(row.get("Filters", "")).split(",") if f.strip()]
        
        # Explode columns
        for col in [c.strip() for c in str(row.get("Columns", "")).split(",") if c.strip()]:
            table = col.split(".", 1)[0] if "." in col else ""
            rows.append({
                "PBIX File": pbix_filename,
                "Section": section,
                "Section Display": section_display,
                "Visual Index": visual_idx,
                "Type": vtype,
                "Tables": table,
                "iscolumn/ismeasure": "iscolumn",
                "Columns/Measures": col,
                "Filters": ", ".join(filters)
            })
        # Explode measures
        for meas in [m.strip() for m in str(row.get("Measures", "")).split(",") if m.strip()]:
            table = ""
            if "." in meas:
                if "(" in meas and ")" in meas:
                    import re
                    m = re.search(r"\(([^.]+)\.[^)]+\)", meas)
                    table = m.group(1) if m else ""
                else:
                    table = meas.split(".", 1)[0]
            rows.append({
                "PBIX File": pbix_filename,
                "Section": section,
                "Section Display": section_display,
                "Visual Index": visual_idx,
                "Type": vtype,
                "Tables": table,
                "iscolumn/ismeasure": "ismeasure",
                "Columns/Measures": meas,
                "Filters": ", ".join(filters)
            })
    out_df = pd.DataFrame(rows)
    return out_df

def pbix_to_exploded_df(pbix_path):
    pbix_filename = os.path.basename(pbix_path)
    temp_dir = unzip_pbix(pbix_path)
    layout_path = find_layout_json(temp_dir)
    layout_json = read_layout_json(layout_path)
    # Use your extraction function here
    visuals_df = pbix_layout_to_df(layout_path)
    exploded_df = explode_df_one_table_per_row2(visuals_df, pbix_filename)
    # Optionally, clean up temp_dir if needed
    return exploded_df

In [66]:
pbix_file =  'D:\\Project\\pbix\\Human Resources Sample PBIX.pbix'
outfile ="D:\\Project\\pbix\\Layout5.csv"
df = pbix_to_exploded_df(pbix_file)
print(df)
df.to_csv(outfile, index=False)

                           PBIX File         Section      Section Display  \
0   Human Resources Sample PBIX.pbix  ReportSection2            New Hires   
1   Human Resources Sample PBIX.pbix  ReportSection2            New Hires   
2   Human Resources Sample PBIX.pbix  ReportSection2            New Hires   
3   Human Resources Sample PBIX.pbix  ReportSection2            New Hires   
4   Human Resources Sample PBIX.pbix  ReportSection2            New Hires   
..                               ...             ...                  ...   
60  Human Resources Sample PBIX.pbix  ReportSection4  New Hires Scorecard   
61  Human Resources Sample PBIX.pbix  ReportSection4  New Hires Scorecard   
62  Human Resources Sample PBIX.pbix  ReportSection4  New Hires Scorecard   
63  Human Resources Sample PBIX.pbix  ReportSection4  New Hires Scorecard   
64  Human Resources Sample PBIX.pbix  ReportSection4  New Hires Scorecard   

    Visual Index                           Type    Tables iscolumn/ismeasur

In [ ]:
# # Usage:
# infile ="D:\\Project\\pbix\\Layout.json"

# df = pbix_layout_to_df(infile)
# # exploded_df = explode_visuals_df(df)
# exploded_df = explode_df_one_table_per_row2(df,infile)
# print(exploded_df)
# exploded_df.to_csv(outfile, index=False)

                      PBIX File         Section      Section Display  \
0   D:\Project\pbix\Layout.json  ReportSection2            New Hires   
1   D:\Project\pbix\Layout.json  ReportSection2            New Hires   
2   D:\Project\pbix\Layout.json  ReportSection2            New Hires   
3   D:\Project\pbix\Layout.json  ReportSection2            New Hires   
4   D:\Project\pbix\Layout.json  ReportSection2            New Hires   
..                          ...             ...                  ...   
60  D:\Project\pbix\Layout.json  ReportSection4  New Hires Scorecard   
61  D:\Project\pbix\Layout.json  ReportSection4  New Hires Scorecard   
62  D:\Project\pbix\Layout.json  ReportSection4  New Hires Scorecard   
63  D:\Project\pbix\Layout.json  ReportSection4  New Hires Scorecard   
64  D:\Project\pbix\Layout.json  ReportSection4  New Hires Scorecard   

    Visual Index                           Type    Tables iscolumn/ismeasure  \
0              1                      lineChart      Da